In [22]:
import sys
import csv
import os

In [23]:
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction import DictVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

## Data Prepreparation

In [24]:
def sentence_to_object(file):

    text = file.readlines();
    array = list()
    dict = {}

    for line in text:
        data = line.split("\t")
        if data[0] != "?":
            word = data[0];
            pos = data[len(data)-1]
            pos = get_tags(pos, 1)
            dict[word] = pos.strip()
        elif data[0] == '?':
            array.append(dict)
            dict = {}
    
    return array

In [25]:
def get_tags(tag, level):

    if level == 1:
        tag = tag.split("-")[0]
    elif level == 2:
        temp = tag.split("-")[1]
        tag = temp[:2]
    elif level == 3:
        temp = tag.split("-")[1]
        tag = temp[2:]

    return tag

In [26]:
def transform(data):

    dict = {'NN':0, 'PR':0, 'DT':0, 'LM':0, 'CC':0, 'VB':0, 'JJ':0, 'RB':0, 'CD':0, 'TS':0}
    temp = []

    for sentence in data:
        for key in sentence:
            dict[sentence[key]] += 1

        temp.append(dict)
        dict = {'NN':0, 'PR':0, 'DT':0, 'LM':0, 'CC':0, 'VB':0, 'JJ':0, 'RB':0, 'CD':0, 'TS':0}
     

    return temp

In [44]:
def category_to_vector():
    input_file = open(os.path.abspath('files/training_set_category.csv'))
    csv_f = csv.reader(input_file)

    cat = []
    column = 0

    category = {'entity':1, 'abbreviation':2, 'description':3, 'human':4, 'location':5, 'numeric':6}

    for row in csv_f:
        if "Questions" not in row:
            temp = []
            for column in range(1, 2):
                if(row[column] != "Category"):
                    temp.append(category[row[column].lower()])
                elif(len(row[column]) == 0):
                    temp.append(0)
            cat.append(temp)

    return cat

In [50]:
def to_numerical(file):
    input_file = file
    data = []
    
    vector = DictVectorizer(sparse=False)
    data = sentence_to_object(input_file)
    data = transform(data)
#     print "To_numerical"
    print len(data)
    X = vector.fit_transform(data)
    
    return X

In [51]:
data = to_numerical(open(os.path.abspath('files/training_set_pos.out')))
category = category_to_vector()

# print len(data)
print len(category)

X_train, X_test, y_train, y_test = train_test_split(data, category, test_size=0.20, random_state = 42)

2549
1526


ValueError: Found input variables with inconsistent numbers of samples: [2549, 1526]

## Training

Support Vector Machine

classifier = OneVsRestClassifier(LinearSVC(C=1.0, random_state=42))
classifier.fit(data, category)

In [ ]:
testing_data = to_numerical(open(os.path.abspath('files/testingset_pos.out')))
prediction = classifier.predict(testing_data)
accuracy  = classifier.score()